##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Game Design Brainstorming

This cookbook illustrates how you can employ the instruction-tuned model version of Gemma as a chatbot to aid in your game design process by fostering brainstorming sessions effectively.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Game_Design_Brainstorming.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [2]:
import os
from google.colab import userdata

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies

Install Keras and KerasNLP.

In [3]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.1/792.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00


## Core Concept

In [4]:
# A super concise description of your game
pitch = "In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution."

## Load Model

In [5]:
import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

import textwrap
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.compile(sampler="top_k")


class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result

  def show_history(self):
    for h in self.history:
      print(h)


chat = ChatState(gemma_lm)

## Elaborate with the given core game concept

In [6]:
result = chat.send_message(
    f"Elaborate about this game with the given core concept below.\n{pitch}"
)
to_markdown(result)

> Okay, let's flesh out this game idea:
> 
> **Title:** The Clockwork Shogun
> 
> **Genre:** Action RPG with a heavy emphasis on narrative and choice-driven story. 
> 
> **Setting:** 
> 
> * **Joseon Reimagined:**  The game takes place in a Joseon-inspired setting, but with a futuristic twist. Steam-powered technology, clockwork automatons, and intricate gears are woven into everyday life. 
> * **Imperial City of Gyeongju:** The bustling metropolis, a blend of ancient traditions and advanced technology. Think ornate palaces powered by steam, clockwork streetlights, and citizens using mechanical tools for daily tasks.
> * **Outskirts and Hidden Villages:** Explore remote areas with traditional Joseon architecture, hinting at a hidden past, and encounter nomadic tribes using clockwork devices for survival.
> * **The Shogunate:** The ruling power in the Joseon Dynasty is a technologically advanced, almost totalitarian regime, with strict laws and control over the population. The Shogunate has a complex history, with whispers of a forgotten past and hidden agendas.
> 
> 
> **Character & Story:**
> 
> * **You are a young, ambitious inventor:**  You have a knack for building and tinkering with clockwork contraptions. You are a rebel with a desire to uncover the truth about the past. 
> * **The Past is Unraveling:** Your journey begins when an ancient scroll is discovered in your workshop, detailing a forgotten legend of a hidden power source that could change the fate of the nation. 
> * **The Revolution:**  You become embroiled in the rising tide of rebellion against the Shogunate. 
> 
> **Gameplay:**
> 
> * **Action-Packed Battles:** Utilize your clockwork creations as weapons. Think of steampunk-style weapons like clockwork cannons, grappling hooks, or mechanical shields.
> * **Exploration:** Explore the bustling streets, hidden temples, and the vast wilderness, uncovering secrets and interacting with a cast of diverse characters.
> * **Dialogue System:**  Your choices impact the story, leading to multiple endings and different perspectives on the rebellion.
> * **Crafting and Customization:**  Develop your own unique clockwork gadgets, upgrade existing inventions, and customize your character’s gear.
> * **Social and Political Intrigue:** Engage in political intrigue by forming alliances, sabotaging the Shogunate's operations, or exposing their secrets.
> 
> **Themes:**
> 
> * **Technological Advancement vs. Tradition:** Explore the tension between embracing technology and preserving ancient customs within the Joseon context. 
> * **Power and Control:** Examine the dangers of unchecked power and the importance of freedom and individual expression.
> * **The Power of Knowledge:** The game highlights the importance of historical truth and the dangers of censorship and manipulation.
> * **Hope and Resistance:** The narrative explores the resilience of the human spirit against authoritarianism and the fight for a better future.
> 
> **Potential Features:**
> 
> * **Character Development:**  Choose your own path by aligning with different factions, each with unique goals and perspectives.  Develop a strong character with their own backstory.
> * **Moral Choices:**  The game will force the player to make difficult choices. 
> * **Multiple Endings:** Depending on choices and actions, the player can experience a variety of endings.
> * **Stunning Visuals:**  Combine the elegance of Joseon-inspired architecture with the steampunk aesthetic of advanced technologies.
> 
> 
> **This game concept offers a unique blend of action, intrigue and historical fantasy, providing players with a compelling narrative experience in a fascinating world.** 
> <end_of_turn>

## Main Characters

In [7]:
result = chat.send_message("Design main characters.")
to_markdown(result)

> Here's a character design outline for the Clockwork Shogun game:
> 
> **Protagonist:**
> 
> * **Name:** Jin-ah Park (meaning "bright" and "flower" in Korean)
> * **Age:** 22
> * **Background:** A young, passionate inventor and tinkerer. Jin-ah grew up in the shadow of the Shogunate, witnessing its oppressive rule firsthand.  They have a talent for understanding and building intricate clockwork mechanisms and are driven by a desire to uncover the truth behind their family's mysterious past. They are resourceful, quick-witted, and possess a rebellious spirit.
> 
> * **Visual:** Jin-ah is a slender young woman with striking dark eyes and short, choppy black hair. She has a mischievous smile and a determined expression. They often wear practical attire with pockets full of tools, a worn leather satchel, and maybe even a small clockwork automaton as a companion.
> 
> **Supporting Characters:**
> 
> * **Name:** Eun-ji (meaning "graceful and virtuous")
> * **Age:** 25
> * **Background:** A skilled warrior and a trusted member of the rebel movement. Eun-ji is fiercely loyal to Jin-ah and a master of both traditional swordsmanship and the use of advanced clockwork weaponry. They are a strong, silent type, but their loyalty is unwavering and they possess a deep understanding of the complexities of the Joseon court.
> * **Visual:** Eun-ji has a lean, athletic build and long, braided hair tied back in a bun. They are adorned with simple yet elegant armor that seamlessly blends traditional and modern design elements.
> 
> * **Name:** Master Kang (meaning "wise old master")
> * **Age:** 65
> * **Background:** A renowned scholar of Joseon history and a former court advisor. Master Kang is deeply connected with ancient texts and possesses a vast knowledge of forgotten traditions. He serves as Jin-ah's mentor and guide, offering cryptic warnings, historical insights, and cryptic clues about the power source. 
> * **Visual:** Master Kang is a frail but sharp-eyed elder. He wears traditional Hanbok with an air of wisdom and authority, often adorned with ornate jewelry and intricate clockwork accessories.
> 
> **Antagonist:**
> 
> * **Name:** Lord Han (meaning "bright moon")
> * **Age:** 40
> * **Background:**  The ruthless Shogunate Minister, Lord Han is a powerful manipulator with a thirst for power. He believes in a strict, technologically controlled society.  He sees Jin-ah as a threat and will stop at nothing to crush the rebellion. 
> * **Visual:** Lord Han is a tall and imposing man with a cold, calculating demeanor. His clothing is a mixture of traditional and futuristic, with ornate armor that reflects his position of power.
> 
> 
> **Additional Notes:**
> 
> 
> * The characters will have their own unique dialogue, personalities, and motivations, creating a rich tapestry of relationships and dynamics within the game. 
> * The characters' relationships and alliances can be further explored through side missions, quests, and interactions.
> * The characters can be developed further as the game progresses, allowing player choices and actions to shape their destinies.
> 
> 
> 
> Let me know if you'd like to explore any of these characters or relationships in more depth, and we can continue to develop their backstories! 
> <end_of_turn>

## Villain Characters

In [8]:
result = chat.send_message("Design villain characters.")
to_markdown(result)

> Here are some villain concepts to create a compelling and nuanced antagonist for your Clockwork Shogun game:
> 
> **The "Clockwork Tyrant":**
> 
> * **Name:** Lord Shin (meaning "bright" and "sharp" in Korean)
> * **Age:** 45
> * **Background:**  A brilliant but ruthless engineer and the Shogun's most trusted advisor. He believes that true power lies in the control of technology and that traditional values are a hindrance to progress. Lord Shin sees himself as a savior, building a perfect, clockwork utopia for the Joseon Dynasty. 
> * **Motivation and Goals:** 
>     * **Control and Order:**  His ambition is to establish a totalitarian regime where technology reigns supreme. He believes that the chaos of the past, the human flaws of emotions and free will are obstacles to true progress.
>     * **The Power Source**:  He seeks to control a mysterious artifact known as the "Heart of the Dragon", a mythical clockwork heart rumored to hold untold power. He believes this will unlock a hidden level of technology and allow the Joseon Dynasty to become a technological powerhouse.
> * **Visuals:**  Lord Shin is a tall, imposing man with piercing blue eyes and a sharp, predatory demeanor. He wears a dark, sleek suit with intricate clockwork gears and mechanical accents that reflect his obsession with technology. 
> * **Personality:** Cold, calculating, manipulative, charismatic when it suits him, he is driven by an unshakeable belief in his vision. He is a master strategist, capable of manipulating others and using technology to his advantage.
> 
> **The "Forgotten Legacy":**
> 
> * **Name:** Lady Seong (meaning "beautiful" and "wise")
> * **Age:**  Unknown
> * **Background:**  A mysterious figure from a long-lost lineage within the Joseon Dynasty. She is believed to be a descendant of a powerful sorcerer who mastered ancient clockwork magic. She has been hiding in the shadows for centuries, waiting for the right time to return and reshape the world.
> * **Motivation:**  Seong seeks to restore the ancient power of her ancestors, believing they are the key to achieving true enlightenment.  She wants to unleash a new form of technological magic that will usher in a new world order.
> * **Visuals:** Lady Seong is a captivating and enigmatic figure, with a haunting beauty that belies her age. She wears flowing traditional clothing but with intricate clockwork embellishments, her movements fluid and graceful. 
> * **Personality:**  Ambiguous and enigmatic, she can be charming and manipulative. However, her true motives are shrouded in secrecy, and her intentions are unclear, making her a powerful and dangerous adversary.
> 
> **Additional Villain Concepts:**
> 
> *  **The Clockwork Cultist:** This could be a fanatical follower of a forgotten cult that worships the Heart of the Dragon, willing to unleash its destructive power.
> * **The Clockwork Assassin:** A master assassin, skilled with clockwork weapons and gadgets, working directly for Lord Shin to eliminate anyone who opposes him.
> 
> 
> **Key Points:** 
> 
> * **Motivation:** Each villain should have a strong, well-defined motivation, driving their actions and creating conflict.
> * **Connection to Clockwork:** Their motivations should connect to the game's core mechanics, such as the use of clockwork technology or the search for hidden knowledge. 
> * **Visual Design:**  The villains should have distinct visual designs that reflect their personalities, powers, and motivations.
> * **Moral Ambiguity:**  Consider giving villains some complexity, perhaps a sense of tragedy or a hidden agenda, to create a more nuanced and compelling antagonist. 
> 
> 
> 
> Let me know if you'd like to delve deeper into any of these villains, or want to brainstorm more ideas!  <end_of_turn>

## Main Story

In [9]:
result = chat.send_message(
    "Write the main story of this game with an introduction, development, turn, and conclusion."
)
to_markdown(result)

> Here's the potential story outline, with the introduction, development, turn, and conclusion for your Clockwork Shogun game:
> 
> **I. Introduction (Act 1: Whispers of Revolution)**
> 
> * **The Discovery:** The game begins in Gyeongju, the capital of the Joseon Dynasty.  Jin-ah, a young inventor with a rebellious spirit, is working in their workshop. They stumble upon an ancient scroll detailing a forgotten power source hidden somewhere in the Shogunate's archives – a legend called "The Heart of the Dragon". The scroll hints at a time when Joseon was a technological powerhouse, a legacy lost to time.
> * **Seeds of Rebellion:**  Jin-ah's curiosity is piqued, but they also witness firsthand the harshness of the Shogunate regime, the strict laws, and the constant surveillance. This awakens a sense of urgency and injustice within them.
> * **Meeting the Resistance:** Jin-ah meets Eun-ji, a warrior skilled in traditional weapons and clockwork combat, who is a part of the growing resistance movement.  They share a mutual desire for freedom, a shared vision of a brighter future.
> 
> **II. Development (Act 2: Clockwork Awakening)**
> 
> * **Uncovering the Secret:** Jin-ah and Eun-ji begin a dangerous quest to decipher the scroll and uncover the Heart of the Dragon's location. They navigate the Shogunate's complex bureaucracy and infiltrate secret archives, uncovering clues hidden in the past.  Their investigations lead them to the Clockmaker's Guild, a hidden organization of talented artisans who possess knowledge of advanced technologies. The Guild is both a source of information and a potential ally. 
> 
> * **Master Kang and the Past:**  Master Kang reveals a connection to ancient clockwork magic, lost knowledge that can help unlock the Heart. He guides Jin-ah and Eun-ji in understanding the power of their technology and its potential for both good and evil.  
> * **Lord Shin's Schemes:**  Lord Shin, the Shogunate Minister, is aware of the growing rebellion and is determined to crush it. He begins using his own clockwork technology, creating powerful weapons designed for control and suppression. 
> * **First Battles and Choices:** Jin-ah and Eun-ji are forced to engage in their own skirmishes with the Shogunate's guards and other forces loyal to them.  These battles force them to develop their unique combat skills, learn to utilize advanced clockwork gadgets,  and face difficult choices about the morality of their actions.
> 
> **III. The Turning Point (Act 3: Clockwork Rebellion)**
> 
> * **The Heart Awakened:**  Jin-ah and Eun-ji finally locate the Heart of the Dragon. The discovery triggers a cascade of unforeseen events. The Heart activates, causing strange and unpredictable changes to both the world and the people. 
> * **Lord Shin's Power:**  Lord Shin uses the Heart to amplify his control over technology and the Shogunate forces.  The world around Gyeongju begins to be shaped into a more rigid, automated society, under Lord Shin's iron fist.
> * **Lady Seong's Intervention:**  Lady Seong appears, seeking to use the Heart’s power for a different purpose, one that is both ambitious and dangerous. She reveals a hidden connection to the ancient clockwork magic, and her motives are more complex than initially believed. 
> * **A Choice for the Future:**  Jin-ah and Eun-ji are now faced with a critical decision: embrace Lord Shin's technological utopia or fight to restore the balance of power, potentially unleashing forces beyond their understanding, and risking the destruction of the world they know.
> 
> **IV. Conclusion (Act 4: The Price of Progress)** 
> * **Confrontation and Sacrifice:** The climax sees Jin-ah and Eun-ji confronting Lord Shin in a final showdown that involves a choice to embrace the Heart or to find a way to control its power. 
> * **Consequences of Choice:**  The final choice Jin-ah makes shapes their destiny and the fate of the entire Joseon Dynasty. The game ends with a bittersweet victory, with consequences and sacrifices made in the pursuit of freedom and justice.  
> * **The Future of Gyeongju:** The Heart of the Dragon is either used to create a new society, or it's sealed away, leaving the future of Gyeongju and the nation uncertain. 
> 
> 
> **Key Considerations:**
> 
> * **Choices and Consequence:**  The game should emphasize player choices and their impact on the storyline. Each decision will influence Jin-ah's character development and the fate of the world.
> * **Moral Gray Areas:**  The story should explore complex moral dilemmas, forcing players to question the line between right and wrong.
> * **The Power of Knowledge:**  The game should emphasize the importance of knowledge and its potential for both good and evil. 
>  
> **Note**: These details are a foundation. You can adjust and expand on them, adding new characters, locations, and subplots to create a truly unique and compelling narrative. 
> 
> 
> Let me know if you'd like to delve into any of these details further, or if you have any other ideas!<end_of_turn>

## Show the chat history

In [10]:
chat.show_history()

<start_of_turn>user
Elaborate about this game with the given core concept below.
In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution.<end_of_turn>

<start_of_turn>model
Okay, let's flesh out this game idea:

**Title:** The Clockwork Shogun

**Genre:** Action RPG with a heavy emphasis on narrative and choice-driven story. 

**Setting:** 

* **Joseon Reimagined:**  The game takes place in a Joseon-inspired setting, but with a futuristic twist. Steam-powered technology, clockwork automatons, and intricate gears are woven into everyday life. 
* **Imperial City of Gyeongju:** The bustling metropolis, a blend of ancient traditions and advanced technology. Think ornate palaces powered by steam, clockwork streetlights, and citizens using mechanical tools for daily tasks.
* **Outskirts and Hidden Vil

## Clear History

In [11]:
chat.history.clear()

## Expansion Idea

### Marketing Phrase

In [12]:
chat.history.clear()
pitch = "A new steam-powered toothbrush"
result = chat.send_message(
    f"Generate a marketing phrase for the new product below.\n{pitch}"
)
to_markdown(result)

> Here are some marketing phrases for a new steam-powered toothbrush, playing with different angles:
> 
> **Benefit Focus:**
> 
> * **"Blast away plaque, not your smile: Introducing the SteamShine toothbrush."**
> * **"Experience the power of steam cleaning: Your healthiest smile, just a click away."**
> * **"Deep clean. Gentle touch. SteamShine: Your daily smile revolution."**
> * **"Beyond brushing: SteamShine delivers a powerful, hygienic clean."**
> 
> **Unique Selling Proposition:**
> 
> * **"The future of teeth care is here. Introducing SteamShine: The steam-powered revolution."**
> * **"Say goodbye to plaque and hello to SteamShine's revolutionary cleaning power."**
> * **"SteamShine: It's not just a toothbrush, it's a whole new level of oral hygiene."** 
> 
> **Intriguing/Playful:**
> 
> * **"Feel the steam. See the difference. SteamShine - Your smile will thank you."**
> * **"Ready for a breath of fresh air for your mouth?  Meet SteamShine."**
> * **"Get ready to steam roll your dental hygiene with SteamShine."**
> 
> **Remember:**
> 
> * **Target audience:** Consider who your ideal customer is and tailor your phrasing accordingly (e.g., health-conscious individuals, tech-savvy consumers, etc.).
> * **Brand personality:** Reflect your brand's tone and style in the language you use.
> * **Call to action:** Encourage potential customers to learn more or purchase the product. 
> 
> 
> 
> Let me know if you'd like me to explore more specific angles!  <end_of_turn>

### Florist Ideas

In [13]:
chat.history.clear()
pitch = "Universe and shooting stars"
result = chat.send_message(
    f"Generate a florist idea inspired by the concept below, along with suggestions for suitable flowers.\n{pitch}"
)
to_markdown(result)

> ## Florist Idea: Celestial Bloom
> 
> **Concept:**  This florist would be inspired by the beauty and wonder of space, specifically the universe and shooting stars. It's a magical and romantic theme, perfect for special occasions or simply a unique touch of whimsy.
> 
> **Atmosphere:**
> * Soft, ambient lighting with starry accents. 
> * Celestial-themed music (space-related soundtracks or ambient electronica). 
> *  A dark blue or deep purple color scheme, with pops of silver or gold. 
> *  A touch of celestial imagery like constellations or star charts. 
> 
> **Flower Selection:**
> * **Cosmic Elegance:** White orchids, hydrangeas, and lilies symbolize the purity of space and the night sky.
> * **Shooting Star Sparkle:**  Ranunculus and lisianthus, with their vibrant colors and delicate petals, evoke shooting stars.
> * **Galaxy Glow:**  Deep purple and blue anemones, cosmos, and calla lillies add a sense of depth and intrigue. 
> * **Starry Night Blooms:**  Deep blue delphiniums and blue hydrangeas create a captivating night sky effect.
> * **Lunar Charm:**  White roses and moonflower blossoms, representing the gentle beauty of the moon.
> 
> 
> **Arrangement Ideas:**
> * **Starry Night Wreath:**  A circular arrangement of white and blue blooms with twinkling fairy lights.
> * **Galaxy Vase:**  A large vase filled with cosmos, anemones, and other deep purple flowers, accented by silver glitter.
> * **Shooting Star Bouquet:**  A cascading bouquet of ranunculus, lisianthus, and other vibrant flowers, with a star-shaped accent.
> * **Celestial Terrariums:**  Miniature terrariums filled with moss, air plants, and small blooming flowers, creating a miniature galaxy.
> 
> 
> **Special Offers:**
> * "Cosmic Bouquet" - A pre-designed bouquet with a galaxy or starry sky theme.
> * "Stargazer Package" - A selection of flowers with a celestial twist, along with personalized star map and constellation chart.
> * **"Launch Your Romance"**  A romantic bouquet with a personalized message on a celestial-themed card.
> 
> **Marketing:**
> *  Use social media platforms like Instagram and Pinterest to showcase the unique arrangements and create a visually appealing space.
> *  Offer discounts for special occasions like Valentine's Day, anniversaries, and birthdays.
> *  Collaborate with local photographers or artists to showcase the florist's vision and create a captivating brand image.
> 
> 
> 
> By combining stunning floral arrangements with the enchanting theme of the universe and shooting stars, the Celestial Bloom florist can create a memorable and unforgettable experience for every customer. 
> <end_of_turn>

### Food Recipe

In [14]:
chat.history.clear()
pitch = "Cyberpunk Kraken"
result = chat.send_message(
    f"Generate a cooking recipe with the concept below.\n{pitch}"
)
to_markdown(result)

> ## Cybernetic Kraken Chowder
> 
> **A cyberpunk twist on a classic, with a hint of tentacled terror!**
> 
> **Concept:** This dish blends the hearty goodness of a classic chowder with cyberpunk-inspired twists and futuristic presentation.  
> 
> **Ingredients:** (serves 4-6)
> 
> * **Base:** 
>     * 1 lb. diced smoked sausage (pork, chorizo, or kielbasa)
>     * 1 large onion, chopped 
>     * 2 cloves garlic, minced
>     * 1 tbsp olive oil
>     * 1/2 cup diced bacon, cooked and crumbled
>     * 1 tbsp vegetable broth powder
>     * 4 cups water
> * **Seafood:**
>     * 1 cup diced smoked fish (cod, haddock, or salmon)
>     * 1/2 cup canned mussels, drained and chopped
>     * 1/2 cup frozen shrimp, thawed and chopped
> * **Veggies:**
>     * 1 cup potatoes, diced
>     * 1 can (14 oz) diced tomatoes, undrained
>     * 1/2 cup chopped celery
>     * 1/4 cup chopped green bell pepper
>     * 1/4 cup frozen spinach, thawed and chopped
> * **Cyberpunk Touches:**
>     * 1/4 cup "cyber-sauce" (a spicy, smoky tomato-based sauce, see recipe below)
>     * Edible glitter for garnish
>     * Glowing LED light sticks for a cyberpunk ambiance
> 
> **Cyber-Sauce Recipe:**
> 
> * **Ingredients:**
>     * 1/2 cup canned tomato sauce
>     * 1 tbsp smoked paprika
>     * 1 tsp chili powder
>     * 1/2 tsp cumin
>     * A pinch of black pepper
>     * A dash of liquid smoke (optional)
> * **Instructions:** Combine all ingredients in a saucepan and simmer for 10 minutes.
> 
> **Instructions:**
> 1. **Cook the base:** In a large saucepan, heat olive oil over medium heat. Add onion and cook until translucent, about 5 minutes. Add garlic and cook for 1 minute more. 
> 2. Add diced sausage and bacon. Cook until sausage is crispy. Stir in vegetable broth powder and water. 
> 3. Bring to a boil, then reduce heat and simmer for 10 minutes.
> 4. **Add seafood and veggies:** Add diced smoked fish, mussels, shrimp, potatoes, tomatoes, celery, and green pepper to the soup. Simmer for 15-20 minutes or until vegetables are tender. 
> 5. **Add cyber-sauce:**  Stir in cyber-sauce and cook for 1 minute. 
> 6. **Serve:** Ladle soup into bowls. Decorate with edible glitter and LED light sticks for a cyberpunk glow. 
> 
> **Serving Suggestion:**  Serve with crusty bread for dipping and a side of neon-colored salad for a truly immersive experience.
> 
> 
> **Enjoy your Cybernetic Kraken Chowder, and let your taste buds explore the depths of the digital age!**
> 
> 
> **Bonus Tip:** To add a visual element, create "cyber-circuits" out of edible food coloring and arrange them around the rim of the bowls. 
> <end_of_turn>